In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
data = pd.read_csv('Churn_modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [4]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
label_encoder = LabelEncoder()  
data['Gender'] = label_encoder.fit_transform(data['Gender'])

In [6]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [7]:
from sklearn.preprocessing import OneHotEncoder

In [8]:
ohe = OneHotEncoder(sparse_output=False)
geography_encoded = ohe.fit_transform(data[['Geography']])


In [9]:
geography_encoded

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [10]:
geo_encoded_df=pd.DataFrame(geography_encoded, columns=ohe.get_feature_names_out(['Geography']))

In [11]:
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [12]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [13]:
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

with open('one_hot_encoder.pkl', 'wb') as f:
    pickle.dump(ohe, f)


In [14]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [15]:
X=data.drop('Exited', axis=1)
y=data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [16]:
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

ANN Implementation

In [17]:
import tensorflow as tf

In [18]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout  
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [19]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  #1st input layer
    Dense(32, activation='relu'),  #2nd hidden layer
    Dense(1, activation='sigmoid')  #output layer
])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [23]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [24]:
history = model.fit(
    X_train,y_train,validation_data = (X_test,y_test), epochs=100,
    callbacks=[early_stopping_callback, tensorboard_callback]
)

Epoch 1/100


250/250 [==============================] - 3s 5ms/step - loss: 0.4547 - accuracy: 0.8025 - val_loss: 0.3987 - val_accuracy: 0.8280
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3898 - accuracy: 0.8375 - val_loss: 0.3612 - val_accuracy: 0.8520
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3603 - accuracy: 0.8530 - val_loss: 0.3481 - val_accuracy: 0.8530
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3458 - accuracy: 0.8566 - val_loss: 0.3441 - val_accuracy: 0.8605
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3404 - accuracy: 0.8586 - val_loss: 0.3447 - val_accuracy: 0.8555
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3363 - accuracy: 0.8594 - val_loss: 0.3425 - val_accuracy: 0.8605
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3336 - accuracy: 0.8605 - val_loss: 0.3397 - val_accuracy: 0.85

In [25]:
model.save('model.h5')

c:\Users\7ein\OneDrive\Desktop\ann\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
%load_ext tensorboard

In [27]:
%tensorboard --logdir logs/fit/20250917-225322

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\7ein\AppData\Local\Programs\Python\Python311\Scripts\tensorboard.exe\__main__.py", line 4, in <module>
  File "C:\Users\7ein\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorboard\main.py", line 27, in <module>
    from tensorboard import default
  File "C:\Users\7ein\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorboard\default.py", line 40, in <module>
    from tensorboard.plugins.hparams import hparams_plugin
  File "C:\Users\7ein\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorboard\plugins\hparams\hparams_plugin.py", line 30, in <module>
    from tensorboard.plugins.hparams import backend_context
  File "C:\Users\7ein\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorboard\plugins\hparams\backen